In [14]:
# Sean Meisch Numerical Methods Final

import pandas as pd
import numpy as np
from sympy import symbols, diff
from scipy.integrate import quad
from sympy import Symbol, Derivative
import matplotlib.pyplot as plot


t_wall = 10 # C
t_window = -23 # C

Q = 80

qpos = [4,5,8,10,14] # biased towards wall side ; most effecient arrangement

# Bring in data from Ditmesh #

t = np.matrix(pd.read_excel(r'Element_Nodes_T_Simp.xlsx'))
print(t)
for i in range(0,len(t)):
    for j in range(0,3):
        t[i,j] -= 1
#print(t)

p = np.matrix(pd.read_excel(r'Position_P_Simp.xlsx'))
print (p)

edgenodes = np.matrix(pd.read_excel(r'Edge_Nodes_Simp.xlsx'))

for i in range(0,len(edgenodes)):
    edgenodes[i] -= 1
    
edgepos = np.matrix(pd.read_excel(r'Edge_Position_Pv_Simp.xlsx'))

[[ 9  6  8]
 [ 7  9 10]
 [ 6  9  7]
 [ 5  3  1]
 [ 6  3  5]
 [ 8  6  5]
 [ 4  3  6]
 [ 6  7  4]
 [ 2  3  4]]
[[  7.           8.        ]
 [ 10.         294.        ]
 [ 96.05358759 143.1637938 ]
 [193.58511024 293.63789964]
 [218.77184444   8.41523861]
 [329.36744706 109.26889921]
 [350.81378139 293.32778482]
 [517.           9.        ]
 [517.         149.47042221]
 [517.         293.        ]]


In [20]:


## The boundary conditions are inputted into the unkowntemps vector. There are 4 nodes in this problem, 3 are
# on the boundary and one in the center. The center node is the only unknown in this case

# f = np.array([[(2/3),(2/3),1,(2/3)]]).T # forcing functions
# print(forcing_function)

# unknown_temps = np.array([[0,1,u,0]]).T
# print(unknown_temps)

# #############################
# n is the number of nodes in the mesh

# ###########
#pos = np.matrix([[1.315,2.27],[0,0],[1.315,.76],[2.63,0]]) # xy coordinates of nodes 1-4
#node_pos = np.matrix([[0,1,2],[2,1,3],[0,2,3]]) # elements 1 2 and 3 with the nodes ordered counterclockwise

pos = p
node_pos = t


# print(node_pos)
n = len(p)
P = np.zeros([3,3])
print(n)

# p is a representation of the basis function for each node
def basisP(xyposition, nodepos, element):
    for i in range(0,3):
        P[i,0] = xyposition[nodepos[element,i],0]
        P[i,1] = xyposition[nodepos[element,i],1]
        P[i,2] = 1
    return P

#Q=2

#the next step in the process is to set up a matrix of coeffecients for the matrix representing the phi function

def basis_matrix(xyposition,nodepos,element,P):
    a = 1/np.linalg.det(P) # + Q
    B = []
    x = [xyposition[nodepos[element,0],0]]
    x1 = [xyposition[nodepos[element,1],0]]
    x2 = [xyposition[nodepos[element,2],0]]
    y = [xyposition[nodepos[element,0],1]]
    y1 = [xyposition[nodepos[element,1],1]]
    y2 = [xyposition[nodepos[element,2],1]]
    x = x[0]
    #print(x)
    x1 = x1[0]
    x2 = x2[0]
    y = y[0]
    y1 = y1[0]
    y2 = y2[0]
    B1 = [a*(y1-y2),a*(x2-x1),a*(x1*y2-x2*y1)] #B1 ,B2 ,and B3 creates the coeffecients of phi 1, 2 and 3
    B2 = [a*(y2-y),a*(x-x2),a*(x2*y1-x1*y)]
    B3 = [a*(y-y1),a*(x1-x),a*(x*y1-x1*y)]
    B.append(B1)
    B.append(B2)
    B.append(B3)
    return B

# taking the results of the B matrix , the phi matrix multiplies the coeffecients of the phi matrix by x,y, and 1 and the resulting
# matrix is differentiated in terms of x and y

def phi_matrix(B): # The phi matrix calculates the value of the phi function at different nodes
    x = Symbol('x') # create symbolic variables
    y = Symbol('y')
    O = []
    O.append(x)
    O.append(y)
    O.append(1)
    f = np.dot(B,O)
    phix = (diff(f[0],x)) # differentiate the matrix
    phiy = (diff(f[0],y))
    phix1 = (diff(f[1],x))
    phiy1 = (diff(f[1],y))
    phix2 = (diff(f[2],x))
    phiy2 = (diff(f[2],y))
    phi = []
    phi1 = [phix,phiy]
    phi2 = [phix1,phiy1]
    phi3 = [phix2,phiy2]
    phi.append(phi1)
    phi.append(phi2)
    phi.append(phi3)
    return phi


def area(xyposition,nodepos,element): # Calculates the area of the elements which are triangles
    x = [xyposition[nodepos[element,0],0]]
    x1 = [xyposition[nodepos[element,1],0]]
    x2 = [xyposition[nodepos[element,2],0]]
    y = [xyposition[nodepos[element,0],1]]
    y1 = [xyposition[nodepos[element,1],1]]
    y2 = [xyposition[nodepos[element,2],1]]
    X1 = x
    X2 = x1
    X3 = x2
    Y1 = y
    Y2 = y1
    Y3 = y2
    X = []
    Y = []
    X.append(X1)
    X.append(X2)
    X.append(X3)
    Y.append(Y1)
    Y.append(Y2)
    Y.append(Y3)
    
    area = ((X1[0]*(Y2[0]-Y3[0]))+(X2[0]*(Y3[0]-Y1[0]))+(X3[0]*(Y1[0]-Y2[0])))/2
    
    elementarea = area
    
    return elementarea


## each element has a local coeffecient matrix associated with it

def assemble_local(phi,elementarea):
    #phi = np. asarray (phi)
    A = np.zeros([3,3])
    for i in range(0,3):
        for j in range(0,3):
            A[i][j] = (phi[j][0]*phi[i][0]+phi[j][1]*phi[i][1])*elementarea
    return A



def assemble_global(phi,elementarea,element,nodepos,n):
    A_global = np.zeros([n,n])
    for i in range(0,3):
        for j in range(0,3):            
            A_global[nodepos[element,i],nodepos[element,j]] += (phi[j][0]*phi[i][0]+phi[j][1]*phi[i][1])*elementarea
    return A_global

A_global = np.zeros([n,n])

for i in range(0,n-1):
    #P = np.zeros ([3,3])
    P = basisP(pos,node_pos, i)
    B = basis_matrix(pos,node_pos,i,P)
    phi = phi_matrix(B)
    elementarea = area(pos,node_pos,i)
    #A = assemble_local(phi,elementarea)
    A_global += assemble_global(phi,elementarea,i,node_pos,n)
    # A_global = np.negative(A_global)

    


10


In [23]:
A_local = np.zeros ([3,3])
A_global = np.zeros([n,n])

for i in range(0,n-1):
    P = basisP(pos,node_pos, i)
    B = basis_matrix(pos,node_pos,i,P)
    phi = phi_matrix(B)
    elementarea = area(pos,node_pos,i)
        #A = assemble_local(phi,elementarea)
    A_local = assemble_local(phi,elementarea)
    for j in range(0,3):
        for k in range(0,3):
            A_global[node_pos[i,j],node_pos[i,k]] += A_local[]
        # A_global = np.negative(A_global)
    
print("global length", len(A_global))
print(n)

print(np.linalg.det(A_global))

global length 10
10
1.56610833032179e-15
